
1) Create a dataset that contains the monthly means of Sea Surface Temperature anomalies and total column water vapor from Jan 1979-Dec 2024 over the Pacific Basin (65°N to 65°S, 120°E to 60°W) masked out over land - save this to your computer. The data and land sea mask is available here: [https://cds.climate.copernicus.eu/datasets/reanalysis-era5-single-levels-monthly-means?tab=download](https://cds.climate.copernicus.eu/datasets/reanalysis-era5-single-levels-monthly-means?tab=download)

In [1]:
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
ds = xr.open_dataset("data_stream-moda_stepType-avgua.nc")
ds

<xarray.Dataset> Size: 2GB
Dimensions:     (valid_time: 552, latitude: 521, longitude: 721)
Coordinates:
    number      int64 8B ...
  * valid_time  (valid_time) datetime64[ns] 4kB 1979-01-01 ... 2024-12-01
  * latitude    (latitude) float64 4kB 65.0 64.75 64.5 ... -64.5 -64.75 -65.0
  * longitude   (longitude) float64 6kB -60.0 -59.75 -59.5 ... 119.5 119.8 120.0
    expver      (valid_time) <U4 9kB ...
Data variables:
    sst         (valid_time, latitude, longitude) float32 829MB ...
    lsm         (valid_time, latitude, longitude) float32 829MB ...
    tcwv        (valid_time, latitude, longitude) float32 829MB ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-10-24T14:35 GRIB to CDM+CF via cfgrib-0.9.1...

In [3]:
# mask out over land
# first create mask of land/ocean
mask = ds['lsm'] < 0.5

# and now apply mask to data
ds_masked = ds.copy()
ds_masked['sst']  = ds['sst'].where(mask)
ds_masked['tcwv'] = ds['tcwv'].where(mask)

In [4]:
# now to calculate the anomolies of SST 
# we want to calculate the monthly average SST over this time period
sst_clim = ds_masked['sst'].groupby('valid_time.month').mean('valid_time')

In [5]:
# and the anomaly value will be the difference between the actual months value and 
# the long-term average

sst_anom = ds_masked['sst'].groupby('valid_time.month') - sst_clim
ds_masked['sst_anom'] = sst_anom

In [6]:
# check work
print(ds_masked)

<xarray.Dataset> Size: 3GB
Dimensions:     (valid_time: 552, latitude: 521, longitude: 721)
Coordinates:
    number      int64 8B ...
  * valid_time  (valid_time) datetime64[ns] 4kB 1979-01-01 ... 2024-12-01
  * latitude    (latitude) float64 4kB 65.0 64.75 64.5 ... -64.5 -64.75 -65.0
  * longitude   (longitude) float64 6kB -60.0 -59.75 -59.5 ... 119.5 119.8 120.0
    expver      (valid_time) <U4 9kB ...
    month       (valid_time) int64 4kB 1 2 3 4 5 6 7 8 9 ... 5 6 7 8 9 10 11 12
Data variables:
    sst         (valid_time, latitude, longitude) float32 829MB 271.5 ... 272.0
    lsm         (valid_time, latitude, longitude) float32 829MB 0.0 0.0 ... 0.0
    tcwv        (valid_time, latitude, longitude) float32 829MB 4.23 ... 8.22
    sst_anom    (valid_time, latitude, longitude) float32 829MB -0.01761 ... ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:    

2)  From the dataset shown in 1, compute anomalies by deseasonalizing the data (remove the mean monthly anomaly from the annual mean from each point) and detrend the data. Then, standardize the SST anomalies.  Standardization means subtract the long term mean and divide by the standard deviation, which can be accomplished using the methods described here: [6.3. Preprocessing data &mdash; scikit-learn 1.1.2 documentation](https://scikit-learn.org/stable/modules/preprocessing.html)

In [7]:
# define functions for code chunks given in assignment instructions

def _time_as_float(time: xr.DataArray, time_dim: str) -> xr.DataArray:
    # numeric seconds since first timestamp (keeps numbers small)
    return (time - time.isel({time_dim: 0})).astype("timedelta64[s]").astype("int64").astype("float64")

def linear_detrend(obj: xr.DataArray | xr.Dataset, time_dim: str = "time") -> xr.DataArray | xr.Dataset:
    """
    Remove a linear trend y ~ s*(t - t̄_valid) + ȳ_valid at each grid point.
    Closed-form LS using reductions; dask-friendly; handles NaNs.
    """
    t = _time_as_float(obj[time_dim], time_dim)  # (time,)
    def _detrend_da(da: xr.DataArray) -> xr.DataArray:
        da = da.sortby(time_dim).astype("float32")
        if hasattr(da.data, "chunks"):
            da = da.chunk({time_dim: -1})  # one chunk along time
        mask = da.notnull()                                # (time, ...)
        t_b = t.broadcast_like(da)                         # (time, ...)
        t_mean_valid = t_b.where(mask).mean(time_dim, skipna=True)
        tc = t_b - t_mean_valid                            # centered time per point
        num = (da * tc).sum(time_dim, skipna=True)
        den = (tc**2).sum(time_dim, skipna=True)
        slope = xr.where(den > 0, num / den, 0.0)
        ybar  = da.mean(time_dim, skipna=True)
        trend = slope * (t_b - t_mean_valid) + ybar
        return (da - trend).astype("float32")
    return obj.map(_detrend_da) if isinstance(obj, xr.Dataset) else _detrend_da(obj)

def monthly_anom_and_z(
    detr: xr.DataArray | xr.Dataset,
    time_dim: str = "time",
    base_period: tuple[str, str] | None = None,
    ddof: int = 1,
    eps: float = 1e-6,
):
    """
    From linearly-detrended data, remove monthly climatology and compute monthly z-scores.
    Returns (anom, z). Works for Dataset or DataArray.
    """
    clim_src = detr if base_period is None else detr.sel({time_dim: slice(*base_period)})
    key = f"{time_dim}.month"

    clim_mean = clim_src.groupby(key).mean(time_dim, skipna=True)
    anom = detr.groupby(key) - clim_mean

    clim_std = clim_src.groupby(key).std(time_dim, skipna=True, ddof=ddof)
    safe_std = xr.where(clim_std > eps, clim_std, np.nan)
    z = anom.groupby(key) / safe_std
    return anom, z

In [9]:
ds1 = ds_masked.chunk({"valid_time": -1})

detr = linear_detrend(ds1, time_dim="valid_time")
anom, z = monthly_anom_and_z(detr, time_dim="valid_time") 

print(type(z), list(z.data_vars))   

<class 'xarray.core.dataset.Dataset'> ['sst', 'lsm', 'tcwv', 'sst_anom']


In [10]:
sst_coarse1 = z.isel(latitude=slice(None, None, 20),
                               longitude=slice(None, None, 20),
                               valid_time=slice(0, 48))
sst_coarse1

<xarray.Dataset> Size: 770kB
Dimensions:     (valid_time: 48, latitude: 27, longitude: 37)
Coordinates:
    number      (valid_time) int64 384B 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0
  * valid_time  (valid_time) datetime64[ns] 384B 1979-01-01 ... 1982-12-01
  * latitude    (latitude) float64 216B 65.0 60.0 55.0 ... -55.0 -60.0 -65.0
  * longitude   (longitude) float64 296B -60.0 -55.0 -50.0 ... 110.0 115.0 120.0
    expver      (valid_time) <U4 768B '0001' '0001' '0001' ... '0001' '0001'
    month       (valid_time) int64 384B 1 2 3 4 5 6 7 8 9 ... 5 6 7 8 9 10 11 12
Data variables:
    sst         (valid_time, latitude, longitude) float32 192kB dask.array<chunksize=(1, 27, 37), meta=np.ndarray>
    lsm         (valid_time, latitude, longitude) float32 192kB dask.array<chunksize=(1, 27, 37), meta=np.ndarray>
    tcwv        (valid_time, latitude, longitude) float32 192kB dask.array<chunksize=(1, 27, 37), meta=np.ndarray>
    sst_anom    (valid_time, latitude, longitude) float32 192kB dask.array<chunksize=(1, 27, 37), meta=np.ndarray>

In [11]:
# standardizing data by subtracting the long term mean and divide by the standard deviation

sst_mean = anom['sst_anom'].mean(dim='valid_time', skipna=True)
sst_std = anom['sst_anom'].std(dim='valid_time', skipna=True)
sst_stdzd = (anom['sst_anom'] - sst_mean) / sst_std
sst_stdzd

<xarray.DataArray 'sst_anom' (valid_time: 552, latitude: 521, longitude: 721)> Size: 829MB
dask.array<truediv, shape=(552, 521, 721), dtype=float32, chunksize=(1, 521, 721), chunktype=numpy.ndarray>
Coordinates:
    number      (valid_time) int64 4kB 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0
  * valid_time  (valid_time) datetime64[ns] 4kB 1979-01-01 ... 2024-12-01
  * latitude    (latitude) float64 4kB 65.0 64.75 64.5 ... -64.5 -64.75 -65.0
  * longitude   (longitude) float64 6kB -60.0 -59.75 -59.5 ... 119.5 119.8 120.0
    expver      (valid_time) <U4 9kB '0001' '0001' '0001' ... '0001' '0001'
    month       (valid_time) int64 4kB 1 2 3 4 5 6 7 8 9 ... 5 6 7 8 9 10 11 12

3) Perform an EOF analysis on the SST anomalies and plot a map of the first 5 EOFs, following M04N05.

In [16]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature
!pip install eofs
from eofs.xarray import Eof
from eofs.examples import example_data_path

In [17]:
# and attempt to reduce resolution/file size so my server will stop crashing :)
sst_coarse1 = sst_stdzd.isel(latitude=slice(None, None, 30),
                               longitude=slice(None, None, 30),
                               valid_time=slice(0, 48))

In [ ]:
sst_coarse = sst_coarse1['sst_anom'].load()

In [ ]:
# I can't run solver on a dask array, so loading into a data array
sst_coarse = sst_coarse1.compute()

In [18]:
solver = Eof(sst_coarse1)

TypeError: the input must be an xarray DataArray

In [ ]:
# Create an EOF solver to do the EOF analysis. Square-root of cosine of
# latitude weights are applied before the computation of EOFs.
coslat = np.cos(np.deg2rad(sst_coarse.coords['latitude'].values))
wgts = np.sqrt(coslat)[..., np.newaxis]
solver = Eof(sst_coarse, weights=wgts)

In [ ]:
# Retrieve the leading EOF, expressed as the correlation between the leading
# PC time series and the input SST anomalies at each grid point, and the
# leading PC time series itself.
eof1 = solver.eofsAsCorrelation(neofs=5)
pc1 = solver.pcs(npcs=5, pcscaling=1)


4) Plot the percent of variance explained by the first 10 EOFs.

In [ ]:
varfrac = solver.varianceFraction()
varpercent = varfrac*100

In [ ]:
# Plot the fraction of variance explained by each EOF
plt.figure(figsize=(11,6))
eof_num = range(1, 11)
plt.plot(eof_num, varpercent[0:10], linewidth=2)
plt.plot(eof_num, varpercent[0:10], linestyle='None', marker="o", color='r', markersize=8)
plt.axhline(0, color='k')
plt.xticks(range(1, 16))
plt.title('Percentage of the total variance represented by each EOF')
plt.xlabel('EOF #')
plt.ylabel('Variance Percent')
plt.xlim(1, 10)
plt.ylim(0, np.max(varfrac)+0.01)

5) Reconstruct the SST field using the first 5 EOFs and plot a map of the Pearson's correlation coefficient ([xarray.corr](https://docs.xarray.dev/en/stable/generated/xarray.corr.html)) of the reconstructed monthly time series and the "observed" SST time series.

In [ ]:
reconstruction = solver.reconstructedField(5)
reconstruction

In [ ]:
clevs = np.linspace(-3, 3, 21)
ax = plt.axes(projection=ccrs.PlateCarree(central_longitude=190))
fill = reconstruction.sel(time='1998-01-15', method='nearest').plot.contourf(ax=ax, levels=clevs, cmap=plt.cm.RdBu_r,
                             add_colorbar=False, transform=ccrs.PlateCarree())
ax.add_feature(cfeature.COASTLINE, color='k', edgecolor='k')
cb = plt.colorbar(fill, orientation='horizontal')
cb.set_label('SST Anomalies (reconstructed)', fontsize=12)

6) Compute a map of the Pearson's correlation coefficient between SST EOF1 and monthly mean detrended, deseasonalized, and standardized monthly mean column water vapor anomalies (don't mask these over land for the plot).  See anything interesting?